In [7]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes
import os
import asyncio
import random
import datetime

#dictionary to store admins ID's
ADMIN_IDS = {}

#list of username = icpeople / username = ChuvaverdeVIP
ADMIN_USERNAMES = ["icpeople", "ChuvaverdeVIP"]

#dictionary to store vip subcriptions and their subscription dates
VIP_SUBSCRIPTIONS = {}

# function to add a vip member
async def add_vip_member(update: Update, context: ContextTypes.DEFAULT_TYPE):
    admin_user = update.message.from_uer
    
    # verify if the user that execute the command is an admin
    if admin_user.username not in ADMIN_USERNAMES:
        await update.message.reply_text("Vocẽ não tem permissão para adicionar membros VIP")
        return
    # args = username and days of subscription
    args = context.args
    
    if len(args) != 2:
        await update.message.reply_text("Uso correto: /adicionar_vip <username> <dias>")
        return
    
    username = args[0]
    days = int(args[1])

    if days not in [30, 90, 180]:
        await update.message.reply_text("Duração de subscrição inválida. Use 30, 90 ou 180 dias.")
        return

    #grab user informations with his username
    user = context.bot.get_chat(username)

    #calculate the actual date and duration
    today = datetime.datetime.now()
    expiration_date = today + datetime.timedelta(days=days)

    VIP_SUBSCRIPTIONS[user.id] = expiration_date

    #add user to vip channel -> ADICIONAR AQUI O ID DO CANAL VIP EM BAIXO(PEDIR A IVAN)
    try:
        await context.bot.add_chat_members(chat_id=VIP_CHANNEL_ID, user_ids = user.id)
        await context.bot.send_message(chat_id = user.id, text=f"Vocẽ foi adicionado ao canal VIP por {days} dias!")
    except EXception as e:
        await update.message.reply_text(f"Erro ao adicionar o usuário ao canal VIP.")
        return

    await update.message.reply_text(f"{username} foi adicionado ao canal VIP por {days} dias. A expiração será em {expiration_date.strftime('%d/%m/%Y')}.")

#function to verify subscriptions that expire soon
async def check_vip_expirations(context: ContextTypes.DEFAULT_TYPE):
    today = datetime.datetime.now().date()
    
    for username, info in VIP_SUBSCRIPTIONS.items():
        expriation_date = info['expiration_date'].date()
        remaining_days = (expiration_date - today).days
        
    if remaing.days == 1:
        #notify the user and the admins
        user_info = await context.bot.get_chat(username)
        admin_notifications = [f"VIP: A subscrição de {user_info.first_name} (@{user_info.username} expira amanhã ({expiration_date.strftime('%d/%m/%Y')}))."]

        for admin_username, admin_id in ADMIN_IDS.items():
            await context.bot.send_message(chat_id = admin_id, text = "\n".join(admin_notifications))
        
        await context.bot.send_message(chat_id= user_info.id, text = f"⚠️ Sua subscrição VIP expira amanhã ({expiration_date.strftime('%d/%m/%Y')}). Não se esqueça de renovar!")

#FAZER AQUI A FUNÇÃO DE REMOVER AUTOMATICAMENTE O USUARIO DA VIP
async def capture_admin_ids(user):
    user = update.message.from_user
    
    # verifiy if the username is in admins list
    if user.username in ADMIN_USERNAMES:
        
        # if it isn't there, add the ussername and id to admin_ids dict
        if user.username not in ADMIN_IDS:
            
            ADMIN_IDS[user.username] = user.id
            await update.message.reply_text(f"ID do administrador {user.username} capturado e armazenado com sucesso.")
        else:
            await update.message.reply_text(f"ID do administrador {user.username} já está armazenado.")


#command to participate in raffle
async def participate(update: Update, context: ContextTypes.DEFAULT_TYPE):
    
    #user
    user = update.message.from_user

    
    #create participants list in context.chat_data
    participants = context.chat_data.setdefault('participants', [])
    
    # add user to participants list if it isn't already in participants list  
    if user.username not in [p["username"] for p in participants]:
        participants.append({
            'first_name' : user.first_name,
            'last_name' : user.last_name,
            'username': user.username,
            'id' : user.id
        })
        
        await update.message.reply_text(f"{user.first_name}, com o username {user.username} está inscrito no sorteio! Obrigado :)")
        
        # armazenar o id do canal chuva verde com context.chat_data -> dicionário do telegram(repositorio)
        context.chat_data['channel_id_chuvaverde_geral'] = update.message.chat_id
        
    else:
        await update.message.reply_text(f"{user.first_name}, vocẽ já está a participar no sorteio. Obrigado")
    
    
    for admin_username, admin_id in ADMIN_IDS.items():
        await context.bot.send_message(chat_id= admin_id, text= "{user.first_name}, com o username {user.username} está inscrito no sorteio!")
    
    #number of participants stored in a variable
    num_participants = len(participants)
    
    # verify if the num of participants are 25,50,100,200, etc , the condition verify if is a power of 2
    if num_participants >= 25 and (num_participants % 25 == 0):
        
        for admin_username, admin_id in ADMIN_IDS.items():
            
            await context.bot.send_message(chat_id= admin_id, text=f"Administrador {admin_username}, o número de participantes é {num_participants}!")
    
    
# command to do the raffle
async def raffle(update: Update, context: ContextTypes.DEFAULT_TYPE):
    
    user = update.message.from_user
    
    # check if the user is an admin. If not, the code makes return and exit the function before running below
    if user.username not in ADMIN_USERNAMES:
        await update.message.reply_text("Vocẽ não tem permissão para usar este comando!")
        return
    
    #grab the participants list from above with context.chat_data dictionary from telegram
    participants = context.chat_data.get('participants', []) #ver análise da linha com pesquina google, mas devolve a lista participants
    
    num_participants = len(participants)
    
    for _, admin_id in ADMIN_IDS.items():
        await context.bot.send_message(chat_id= admin_id, text = f"{num_participants} participantes inscreveram-se no sorteio!")
    
    # grab the channel id stored in dictionary telegram in participate function 
    channel_id_chuvaverde_geral = context.chat_data.get('channel_id_chuvaverde_geral')
    
    if participants:
        winner = random.choice(participants)
        
        winner_id = winner['id']
        
        try:
            user_winner_info = await context.bot.get_chat(winner_id)
            
            # send a messsage to the winner
            await context.bot.send_message(chat_id = winner_id, text= f"📩 🎉 Parabéns {user_winner_info.first_name}! Vocè ganhou o sorteio!")
            
            # send a message to a specific channel group chuva verde geral                        
            await context.bot.send_message(chat_id = channel_id_chuvaverde_geral, text = f" 🎉 O usuário {user_winner_info.first_name} (@{user_winner_info.username}) ganhou o sorteio!")
            
        except Exception as e:
            await update.message.reply_text(f"Erro ao encontrar o vencedor: {str(e)}")
            
        #clean the list after the raffle
        participants.clear()
    else:
        await update.message.reply_text("Ainda não há participantes suficientes no sorteio!")
        

async def vip_information(update: Update, context: ContextTypes.DEFAULT_TYPE):
    
    user = update.message.from_user
    
    #message with vip information
    
    vip_info = (
        "🌟 **Informações VIP** 🌟\n\n"
        "Aqui estão os benefícios de se tornar um membro VIP:\n"
        "1. Acesso exclusivo a sorteios e promoções.\n"
        "2. Suporte prioritário.\n"
        "3. Conteúdo exclusivo apenas para membros VIP.\n"
        "4. Participação em eventos especiais.\n"
        "\nPara mais informações, entre em contato conosco!"
    )
    
    #try to send a private message to user
    
    try:
        await context.bot.send_message(chat_id = user.id, text = vip_info, parse_mode = 'Markdown') #markdown allows to use negrito(**negrito**), italico(_italico_) and links([link](http://example.com))
        await update.message.reply_text(f"{user.first_name}, enviamos uma mensagem privada com as informações VIP!")
    except Exception as e:
        await update.message.reply_text(f"Erro ao enviar mensagem privada: {str(e)}")
        
        
# function that will be called when the start/ command is triggered
async def start(update: Update, context):
    
    # find and save the user first name
    first_name_user = update.message.from_user.first_name
    
    # second name
    seconda_name_user = update.message.from_user.last_name
    
    # username
    username = update.message.from_user.username
    
    # user_id
    user_id = update.message.from_user.id
    
    await update.message.reply_text(f"Olá, {first_name_user} {second_name_user}! O meu nome é Chuvisco! Eu sou o teu bot.")

async def main():
    
    #token variable
    BOT_TOKEN = os.getenv("BOT_TOKEN")
    
    if not BOT_TOKEN:
        print("ERROR: THE BOT TOKEN IS NOT DEFINED!")
    
    # creates bot application
    application = ApplicationBuilder().token(BOT_TOKEN).build()
    
    #add /start button command and associate it with the start function
    start_handler = CommandHandler("começar", start)
    application.add_handler(start_handler)
    
    # add /participar button command 
    participar_handler = CommandHandler("participar", participate)
    application.add_handler(participar_handler)
    
     # add /sortear button command 
    sortear_handler = CommandHandler("sortear", raffle)
    application.add_handler(sortear_handler)
    
    # add /Saber mais sobre VIP button command
    vip_information_handler = CommandHandler("SaberMaisSobreVIP", vip_information)
    application.add_handler(vip_information_handler)
    
    # job para verificação diária, através de job_queue do telegram
    application.job_queue.run_daily(check_vip_expirations, time = datetime.time(hour=9, minute=0)) # agenda o envio da notificação para as 9 da manhã
    
    # starts the bot
    application.run_polling()

